In [1]:
# Setting the model path
PATH_TO_MODEL = "faster_rcnn_resnet101_v1_1024x1024_coco17_tpu-8"
# PATH_TO_MODEL = "ssd_resnet152_v1_fpn_1024x1024_coco17_tpu-8"
#PATH_TO_MODEL = "efficientdet_d3_coco17_tpu-32"
#PATH_TO_MODEL = "efficientdet_d4_coco17_tpu-32"
# PATH_TO_MODEL = "centernet_hg104_1024x1024_coco17_tpu-32"

In [ ]:
!set CUDA_VISIBLE_DEVICES="-1" & python model_main_tf2.py --pipeline_config_path=training/TF2/training/{PATH_TO_MODEL}/pipeline.config --model_dir=training/TF2/training/{PATH_TO_MODEL} --checkpoint_dir=training/TF2/training/{PATH_TO_MODEL} --alsologtostderr

2022-12-27 19:08:50.703737: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
W1227 19:08:51.927455 139850875581248 model_lib_v2.py:1081] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: None
I1227 19:08:51.927534 139850875581248 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I1227 19:08:51.927573 139850875581248 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I1227 19:08:51.927609 139850875581248 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W1227 19:08:51.927658 139850875581248 model_lib_v2.py:1099] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
2022-12-27 19:08:51.937351: I tensorflow/stream_executor/

In [4]:
!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path ./training/TF2/training/{PATH_TO_MODEL}/pipeline.config --trained_checkpoint_dir ./training/TF2/training/{PATH_TO_MODEL}/ --output_directory ./training/TF2/training/{PATH_TO_MODEL}/saved_model/

2022-12-27 11:02:09.661319: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-12-27 11:02:10.734321: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-12-27 11:02:10.769822: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-27 11:02:10.770285: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.755GHz coreCount: 82 deviceMemorySize: 23.70GiB deviceMemoryBandwidth: 871.81GiB/s
2022-12-27 11:02:10.770304: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-12-27 11:02:10.771959: I tensorflow/stream_executor/

<img src='screenies/stats.png'>

This is an example of the stats I was receiving when train my models.

# Faster RCNN ResNet101

<img src='screenies/v1-faster_rcnn_resnet101_scaler.png'>

I have chosen to take an evaluation at 4k epochs as you can see in the last graph (total_loss) convergence begins to happen around 3-3.5k epochs. I also chose to look at an evaluation at 5k epochs as this is where you can see the classification loss converging. I will be saving the 4k eval as this has lower classification losses. It is very difficult to see if the models are overfitting due to an error in the MS COCO eval.

Checkpoint 5 of the model has been selected as the checkpoint is made at exactly 4k epochs.

Precision and Recall 4k epochs

<img src="screenies/precision_and_recall.png">

Precision and Recall 5k epochs

<img src="screenies/v2-faster_rcnn_resnet101_prec_recall.png">

## Detection Examples

<img src="screenies/detection0.png"><img src="screenies/detection1.png"><img src="screenies/detection2.png">

# Testing other model architectures

Before I begin inferencing the model. I want to see how the performance of the other model architectures compare to see if 
a higher performance model can be developed.

### SSD resNet152

<img src="screenies/v1_ssd_scalers_10k.png">

<img src="screenies/v1-ssd-prec-recall.png">

I decided to increase the epochs to 10k for this model as it wasn't seeming to converge, even after the increase it still hasn't converged however I can see that compared to the faster RCNN model above. The performance of the SSD is very poor and presenting loss values much higher than the faster RCNN with no apparent progress towards lower losses.
This is not suprising as the SSD is a very fast detection model usually more focused for tasks that invlove large differencial objects like spotting a car from a van. As we are detecting birds to a sub species level the performance of an SSD model would not be sufficient. 

<img src="screenies/ssd_exampleofmissed.png">

One of the first missed object detections that I have discovered from the example set in the first image. This is an interesting one as from human eye it is extremely obvious to tell that the very bright colours are a bird. It is possible this is a bird not present in the dataset. Further research needed.

### centernet hg104

<img src="screenies/v1-centernet_hg104_scaler1.png">

<img src="screenies/v1-centernet_hg104_prec_recall.png">

A suitable level of convergence to export the model is shown after roughly 6k steps. The epochs of this model had to be increased from 5800 to 8000 as only minor fluctuations were being made to the graph and not enough to draw a suitable level of convergence to reduce overfitting.

## Discussion of Results

After reviewing the training and eval results of each model. I will be focusing my efforts to inferencing the faster RCNN resnet101 model only as I have a limited time frame. The faster RCNN resnet101 produced lower losses on average in most of the measurements including classification loss and localization loss.